In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

In [2]:

# Check if a GPU is available and use it, otherwise use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
RE = "Solar_PBE" 
address = "../data/"

# data_train_csv1 = pd.read_csv(address+RE+'_16.csv', index_col=0)
# data_train_csv2 = pd.read_csv(address+RE+'_17.csv', index_col=0)
# data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
# data_val_csv    = pd.read_csv(address+RE+'_18.csv', index_col=0)
# data_test_csv   = pd.read_csv(address+RE+'_19.csv', index_col=0)

# data_price = pd.read_csv(address+'Price_Elia_Imbalance_16_19.csv', index_col=0)
# data_train_csv['Price(€)'] = data_price['Positive imbalance price'][:len(data_train_csv)]
# data_val_csv['Price(€)']   = data_price['Positive imbalance price'][len(data_train_csv):len(data_train_csv)+len(data_val_csv)]
# data_test_csv['Price(€)']  = data_price['Positive imbalance price'][len(data_train_csv)+len(data_val_csv):]


data_train_csv1 = pd.read_csv(address+RE+'_16.csv', index_col=0)
data_train_csv2 = pd.read_csv(address+RE+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+RE+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+RE+'_19.csv', index_col=0)


data_price = pd.read_csv(address+'Price_Elia_Imbalance_16_19.csv', index_col=0)

# Reset index to default integer indices for proper alignment
data_price_reset = data_price.reset_index(drop=True)
# print(data_price_reset.head(5))   
data_train_csv = data_train_csv.reset_index(drop=True)
data_val_csv = data_val_csv.reset_index(drop=True)
data_test_csv = data_test_csv.reset_index(drop=True)

# Now assign the values with the indices properly aligned
data_train_csv['Price(€)'] = data_price_reset['Positive imbalance price'][:len(data_train_csv)]

val_price = data_price_reset['Positive imbalance price'][len(data_train_csv):len(data_train_csv) + len(data_val_csv)]
val_price = val_price.reset_index(drop=True)
data_val_csv['Price(€)'] =val_price
test_price = data_price_reset['Positive imbalance price'][len(data_train_csv) + len(data_val_csv):] 
test_price = test_price.reset_index(drop=True)
# print(test_price.head(10))
data_test_csv['Price(€)'] = test_price


In [4]:
Battery_Size = 0.15  # p.u.  # Define the battery size as 0.15 per unit (p.u.)
unit = 1  # Set the time unit to 1 (could represent 15 minutes, etc.)

# Calculate the maximum renewable energy capacity from the training, validation, and test datasets
RE_Capacity1 = max(data_train_csv['Power(MW)'])  # Max capacity in training data
RE_Capacity2 = max(data_val_csv['Power(MW)'])    # Max capacity in validation data
RE_Capacity3 = max(data_test_csv['Power(MW)'])   # Max capacity in test data

# Get the maximum price from the price dataset
max_price = max(data_price_reset['Marginal incremental price'])  # Max price for normalizing price data

# Determine the number of units in each dataset
size_train0 = len(data_train_csv) // unit  # Number of units in training data
size_val0 = len(data_val_csv) // unit      # Number of units in validation data
size_test0 = len(data_test_csv) // unit    # Number of units in test data

# Function to normalize power and price data
def normalize_data(power_data, price_data, capacity, max_price, size):
    normalized_power = []  # Initialize list for normalized power data
    normalized_price = []   # Initialize list for normalized price data
    
    # Loop through each time unit
    for i in range(size):
        # Calculate the average power and price for the current time unit and normalize
        power_avg = pd.Series.mean(power_data[i * unit: (i + 1) * unit]) / capacity  # Normalized power
        price_avg = pd.Series.mean(price_data[i * unit: (i + 1) * unit]) / max_price  # Normalized price
        
        # Round the normalized values to 3 decimal places
        power_avg, price_avg = round(power_avg, 3), round(price_avg, 3)
        
        # Only append positive normalized power values to the list
        if power_avg > 0:
            normalized_power.append(power_avg)  # Add to normalized power list
            normalized_price.append(price_avg)    # Add to normalized price list
            
    return normalized_power, normalized_price  # Return normalized power and price lists

# Normalize the training, validation, and test datasets using the defined function
data_train, price_train = normalize_data(data_train_csv['Power(MW)'], data_train_csv['Price(€)'], RE_Capacity1, max_price, size_train0)
data_val, price_val = normalize_data(data_val_csv['Power(MW)'], data_val_csv['Price(€)'], RE_Capacity2, max_price, size_val0)
data_test, price_test = normalize_data(data_test_csv['Power(MW)'], data_test_csv['Price(€)'], RE_Capacity3, max_price, size_test0)


In [6]:
from collections import deque
import random


BUFFER_SIZE = int(1e6)  # Replay buffer size
BATCH_SIZE = 256        # Mini-batch size
GAMMA = 0.99            # Discount factor
TAU = 0.005             # For updating target networks
LR_ACTOR = 3e-4         # Learning rate for actor
LR_CRITIC = 3e-4        # Learning rate for critic
ALPHA = 0.2             # Entropy regularization coefficient
HIDDEN_SIZE = 256       # Size of hidden layers in networks

# Replay Buffer to store experience tuples
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        states, actions, rewards, next_states, dones = zip(*experiences)
        return (torch.FloatTensor(states), torch.FloatTensor(actions),
                torch.FloatTensor(rewards), torch.FloatTensor(next_states),
                torch.FloatTensor(dones))

    def __len__(self):
        return len(self.memory)


In [7]:

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=256):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.mu = nn.Linear(hidden_size, action_dim)
        self.log_std = nn.Linear(hidden_size, action_dim)
    
    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        mu = self.mu(x)
        log_std = self.log_std(x).clamp(-20, 2)
        return mu, log_std

    def sample_action(self, state):
        mu, log_std = self.forward(state)
        std = log_std.exp()
        z = mu + std * torch.randn_like(mu)
        action = torch.tanh(z)
        return action, z, mu, log_std


In [8]:
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_size=256):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.q = nn.Linear(hidden_size, 1)
    
    def forward(self, state, action):
        x = torch.cat([state, action], dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.q(x)


In [16]:
class SACAgent:
    def __init__(self, state_dim, action_dim, gamma=0.99, tau=0.005, lr=3e-4, alpha=0.2):
        self.actor = Actor(state_dim, action_dim).to(device)
        self.critic_1 = Critic(state_dim, action_dim).to(device)
        self.critic_2 = Critic(state_dim, action_dim).to(device)
        self.target_critic_1 = Critic(state_dim, action_dim).to(device)
        self.target_critic_2 = Critic(state_dim, action_dim).to(device)
        
        self.target_critic_1.load_state_dict(self.critic_1.state_dict())
        self.target_critic_2.load_state_dict(self.critic_2.state_dict())
        
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr)
        self.critic_1_optimizer = optim.Adam(self.critic_1.parameters(), lr=lr)
        self.critic_2_optimizer = optim.Adam(self.critic_2.parameters(), lr=lr)
        
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha

    def select_action(self, state):
        state = torch.FloatTensor(state).to(device)
        action, _, _, _ = self.actor.sample_action(state)
        return action.detach().cpu().numpy()

    def update(self, batch):
        state, action, reward, next_state, done = batch
        state = torch.FloatTensor(state).to(device)
        action = torch.FloatTensor(action).to(device)
        reward = torch.FloatTensor(reward).to(device).unsqueeze(1)
        next_state = torch.FloatTensor(next_state).to(device)
        done = torch.FloatTensor(done).to(device).unsqueeze(1)
        
        with torch.no_grad():
            next_action, _, _, log_std = self.actor.sample_action(next_state)
            target_q1 = self.target_critic_1(next_state, next_action)
            target_q2 = self.target_critic_2(next_state, next_action)
            target_q = reward + (1 - done) * self.gamma * (torch.min(target_q1, target_q2) - self.alpha * log_std.exp())
        
        current_q1 = self.critic_1(state, action)
        current_q2 = self.critic_2(state, action)
        
        critic_1_loss = torch.mean((current_q1 - target_q).pow(2))
        critic_2_loss = torch.mean((current_q2 - target_q).pow(2))
        
        self.critic_1_optimizer.zero_grad()
        critic_1_loss.backward()
        self.critic_1_optimizer.step()
        
        self.critic_2_optimizer.zero_grad()
        critic_2_loss.backward()
        self.critic_2_optimizer.step()
        
        new_action, _, _, log_std = self.actor.sample_action(state)
        q1_new = self.critic_1(state, new_action)
        q2_new = self.critic_2(state, new_action)
        q_new = torch.min(q1_new, q2_new)
        
        actor_loss = torch.mean((self.alpha * log_std.exp() - q_new))
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        for param, target_param in zip(self.critic_1.parameters(), self.target_critic_1.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        
        for param, target_param in zip(self.critic_2.parameters(), self.target_critic_2.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


In [10]:
# Environment parameters
E_max   = Battery_Size  # Maximum energy capacity of the battery
P_max   = E_max         # Maximum power output, equal to the maximum energy capacity
tdelta  = unit / 4      # Time step (e.g., 15 minutes if unit is in hours)
soc_min = 0.1          # Minimum state of charge (SOC) for the battery
soc_max = 0.9          # Maximum state of charge (SOC) for the battery

# Coefficients for the battery performance and cost equations
a0 = -1.031; a1 = 35; a2 = 3.685; a3 = 0.2156; a4 = 0.1178; a5 = 0.3201
b0 = 0.1463; b1 = 30.27; b2 = 0.1037; b3 = 0.0584; b4 = 0.1747; b5 = 0.1288
c0 = 0.1063; c1 = 62.49; c2 = 0.0437; d0 = 0.0712; d1 = 61.4; d2 = 0.0288

# Total number of units or capacity in the system (adjust based on configuration)
N = 130 * 215 * E_max / 0.1
beta = 10 / max_price  # A scaling factor based on the maximum price

class Env():
    def __init__(self, data):
        self.data_gen = data[0]  # Data for generation
        self.data_imb = data[1]   # Data for imbalance prices
        self.state = []            # Initialize the state of the environment
 
    def reset(self):
        # Reset the environment to the initial state
        gen = self.data_gen[0]   # Get the first generation value
        imb = self.data_imb[0]   # Get the first imbalance price
        E = E_max / 2            # Initialize the energy state to half the maximum capacity
        state = [[gen, imb, E]]  # Initialize the state with generation, imbalance, and energy
        self.state = state        # Update the state of the environment
        return state              # Return the initial state
 
    def step(self, action):
        index = len(self.state) % len(self.data_gen)
        # Execute a step in the environment based on the given action
        gen = self.data_gen[index]   # Get current generation value based on the state length
        bid = action[0]                         # Bid amount from the action
        rat = action[1]                         # Rate from the action
        imb = self.data_imb[index]   # Get current imbalance price based on the state length

        E = self.state[-1][-1]  # Get the current energy level from the state
        soc = E / E_max         # Calculate state of charge (SOC)

        # Calculate various parameters based on SOC
        Voc = a0 * np.exp(-a1 * soc) + a2 + a3 * soc - a4 * soc**2 + a5 * soc**3  # Open-circuit voltage
        Rs = b0 * np.exp(-b1 * soc) + b2 + b3 * soc - b4 * soc**2 + b5 * soc**3  # Series resistance
        Rts = c0 * np.exp(-c1 * soc) + c2  # Total resistance in the system
        Rtl = d0 * np.exp(-d1 * soc) + d2  # Total leakage resistance
        R = Rs + Rts + Rtl  # Combined resistance

        # Calculate maximum charging and discharging current and power
        I_cmax = 1000000 * (E_max * soc_max - E) / N / (Voc * tdelta)
        I_dmax = 1000000 * (E - E_max * soc_min) / N / (Voc * tdelta)
        p_cmax = N * (Voc * I_cmax + I_cmax**2 * R)  # Maximum charging power
        p_dmax = N * (Voc * I_dmax - I_dmax**2 * R)  # Maximum discharging power

        P_cmax = p_cmax / 1000000  # Convert power to MW
        P_dmax = p_dmax / 1000000  # Convert power to MW

        # Calculate actual charging and discharging power based on bid and generation
        P_c = min(max(rat * (gen - bid), 0), P_max, P_cmax)  # Charging power
        P_d = min(max(rat * (bid - gen), 0), P_max, P_dmax)  # Discharging power

        # Calculate currents based on charging and discharging power
        p_c = 1000000 * P_c / N  # Convert to proper scale
        p_d = 1000000 * P_d / N  # Convert to proper scale

        # Calculate charging and discharging currents using voltage and resistance
        I_c = -(Voc - np.sqrt(Voc**2 + 4 * R * p_c)) / (2 * R)  # Charging current
        I_d = (Voc - np.sqrt(Voc**2 - 4 * R * p_d)) / (2 * R)    # Discharging current
        
        # Update the energy state based on charging/discharging
        if not np.isclose(p_c, 0):  # If charging power is not zero
            eff_c = (Voc * I_c) / p_c  # Calculate charging efficiency
            eff_d = 1                   # Assume discharging efficiency is 1
            E_prime = E + eff_c * P_c * tdelta  # Update energy state after charging
            disp = gen - P_c            # Calculate dispatched generation
        elif not np.isclose(p_d, 0):  # If discharging power is not zero
            eff_d = p_d / (Voc * I_d)  # Calculate discharging efficiency
            eff_c = 1                   # Assume charging efficiency is 1
            E_prime = E - (1 / eff_d) * P_d * tdelta  # Update energy state after discharging
            disp = gen + P_d            # Calculate dispatched generation
        else:  # If neither charging nor discharging
            eff_c = 1; eff_d = 1  # Assume efficiencies are 1
            E_prime = E            # Energy state remains unchanged
            disp = gen             # Dispatch generation remains the same

        # Calculate revenue based on imbalance, dispatched generation, and costs
        revenue = (imb * disp - imb * abs(bid - disp) - beta * (P_c + P_d)) * tdelta

        # Update the next state with current generation, imbalance, and new energy level
        next_state = self.state + [[gen, imb, E_prime]]
        reward = revenue - imb * gen * tdelta  # Calculate reward
        done = False  # Environment is not done yet
        info = [gen, bid, rat, disp, revenue]  # Additional information for debugging
 
        self.state = next_state  # Update the state of the environment
        return next_state, reward, done, info  # Return the next state, reward, done flag, and info

In [17]:
# Initialize environment and agent
state_dim = 3  # e.g., [gen, imb, E] from your environment
action_dim = 2  # e.g., [bid, rat]
agent = SACAgent(state_dim, action_dim)
print(len(data_train))
print(len(price_train))
env_train = Env([data_train, price_train])
# Training parameters
num_episodes = 500
batch_size =128
print_interval=1
max_buffer_size = 5000
replay_buffer = []  # Use a replay buffer to store experiences
max_iteration = len(data_train)  # Maximum number of iterations 
# Initialize lists to store bids, ratios, and revenue metrics for training, validation, and testing
bid_train, bid_val, bid_test = [], [], []
rat_train, rat_val, rat_test = [], [], []
mae_train, mae_val, mae_test = [], [], []
mbe_train, mbe_val, mbe_test = [], [], []
rev_train, rev_val, rev_test = [], [], []

# Define the frequency of updates
update_frequency = 500 # Update every 10 iterations, adjust as needed

for n_epi in range(num_episodes):
    # Initialize lists for the current episode
    bid_train.append([]); bid_val.append([]); bid_test.append([])
    rat_train.append([]); rat_val.append([]); rat_test.append([])
    mae_train.append([]); mae_val.append([]); mae_test.append([])
    mbe_train.append([]); mbe_val.append([]); mbe_test.append([])
    rev_train.append([]); rev_val.append([]); rev_test.append([])

    # Reset the training environment to start a new episode
    state = env_train.reset()
    episode_reward = 0
    iteration_counter = 0  # Counter for iterations since last update

    for t in range(max_iteration):
        action = agent.select_action(state[-1])

        # Verify action dimension
        assert len(action) == 2, f"Action dimension mismatch: expected 2, got {len(action)}"

        next_state, reward, done, info = env_train.step(action)

        # Verify next_state dimension
        assert len(next_state[-1]) == state_dim, f"Next state dimension mismatch: expected {state_dim}, got {len(next_state[-1])}"

        # Verify reward is a scalar
        assert np.isscalar(reward), f"Reward is not a scalar: got {type(reward)}"

        # Verify done is a boolean
        assert isinstance(done, bool), f"Done is not a boolean: got {type(done)}"

        # Verify info is a list with expected length (5 in this case)
        assert isinstance(info, list) and len(info) == 5, f"Info dimension mismatch: expected list of length 5, got {type(info)} with length {len(info)}"

        replay_buffer.append((state[-1], action, reward, next_state[-1], done))
        state = next_state
        episode_reward += reward

        # Unpack information from the environment step
        gen = info[0]; bid = info[1]; rat = info[2]; disp = info[3]; revenue = info[4]
        # Collect training data
        bid_train[n_epi].append(bid)
        rat_train[n_epi].append(rat)
        mae_train[n_epi].append(abs(gen - bid))
        mbe_train[n_epi].append(abs(disp - bid))
        rev_train[n_epi].append(revenue)

        # Increment the iteration counter
        iteration_counter += 1

        if len(replay_buffer) > max_buffer_size:
            replay_buffer.pop(0)


        # Update the agent if there are enough experiences and the counter reaches the update frequency
        if len(replay_buffer) > batch_size and iteration_counter >= update_frequency:
           # print("Updating batch at iteration =", t)
            batch = zip(*[replay_buffer[i] for i in np.random.choice(len(replay_buffer), batch_size, replace=False)])
            agent.update(batch)
            iteration_counter = 0  # Reset counter after update

        if done:
            break


    # Print metrics every 'print_interval' episodes
    if (n_epi + 1) % print_interval == 0:
        MAE_train = round(100 * np.mean(mae_train[n_epi]), 2)  # Mean Absolute Error for training
        MAE_val = round(100 * np.mean(mae_val[n_epi]), 2)    # Mean Absolute Error for validation
        MAE_test = round(100 * np.mean(mae_test[n_epi]), 2)   # Mean Absolute Error for testing
        MBE_train = round(100 * np.mean(mbe_train[n_epi]), 2)   # Mean Bias Error for training
        MBE_val = round(100 * np.mean(mbe_val[n_epi]), 2)     # Mean Bias Error for validation
        MBE_test = round(100 * np.mean(mbe_test[n_epi]), 2)    # Mean Bias Error for testing
        REV_train = round(max_price * RE_Capacity1 * np.mean(rev_train[n_epi]), 3)  # Revenue for training
        REV_val = round(max_price * RE_Capacity2 * np.mean(rev_val[n_epi]), 3)    # Revenue for validation
        REV_test = round(max_price * RE_Capacity3 * np.mean(rev_test[n_epi]), 3)   # Revenue for testing

        # Print the results for the current episode
        print("episode: {}".format(n_epi + 1))
        print("MAE_train: {}%".format(MAE_train).ljust(25), end="")
       # print("MAE_val: {}%".format(MAE_val).ljust(25), end="")
       # print("MAE_test: {}%".format(MAE_test).ljust(25))
        print("MBE_train: {}%".format(MBE_train).ljust(25), end="")
       # print("MBE_val: {}%".format(MBE_val).ljust(25), end="")
       # print("MBE_test: {}%".format(MBE_test).ljust(25))
        print("REV_train: ${}".format(REV_train).ljust(25), end="")
      #  print("REV_val: ${}".format(REV_val).ljust(25), end="")
      #  print("REV_test: ${}".format(REV_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

34631
34631
episode: 1
MAE_train: 38.6%         MBE_train: 39.22%        REV_train: $-82.531      ------------------------------------------------------------------------------------------
episode: 2
MAE_train: 22.33%        MBE_train: 21.23%        REV_train: $-22.03       ------------------------------------------------------------------------------------------
episode: 3
MAE_train: 24.46%        MBE_train: 23.7%         REV_train: $-45.93       ------------------------------------------------------------------------------------------
episode: 4
MAE_train: 24.27%        MBE_train: 23.75%        REV_train: $-47.702      ------------------------------------------------------------------------------------------
episode: 5
MAE_train: 23.85%        MBE_train: 23.49%        REV_train: $-44.398      ------------------------------------------------------------------------------------------
episode: 6
MAE_train: 22.84%        MBE_train: 22.51%        REV_train: $-36.564      -----------------

KeyboardInterrupt: 